In [1]:
"""
Created on Tue Aug 27 15:19:11 2024

@author: Thierry ALLEM

Applications d'algorithmes de Machine Learning supervisés dans le but de prévoir les niveaux d'échanges physiques (import/export d'électricité entres régions limitrophes)
en France métropolitaine sur la période 2020-2023.
L'échantillon d'entrainement est constitué des valeurs relevées sur la période 2013 à 2019.

J'ai testé et comparé 2 méthodologie de Machine Learning: 
-La première en utilisant les données sur l'ensemble des régions (modélisation"globale")
-La seconde en utilsant les données filtrées par région (mmodélisation "par segmentation")

"""

'\nCreated on Tue Aug 27 15:19:11 2024\n\n@author: Thierry ALLEM\n\nApplications d\'algorithmes de Machine Learning supervisés dans le but de prévoir les niveaux d\'échanges physiques (import/export d\'électricité entres régions limitrophes)\nen France métropolitaine sur la période 2020-2023.\nL\'échantillon d\'entrainement est constitué des valeurs relevées sur la période 2013 à 2019.\n\nJ\'ai testé et comparé 2 méthodologie de Machine Learning: \n-La première en utilisant les données sur l\'ensemble des régions (modélisation"globale")\n-La seconde en utilsant les données filtrées par région (mmodélisation "par segmentation")\n\n'

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importation du fichier eco2mix_regional_def préparé, incluant les TCO, TCH, cmax_rte et les populations régionales
# Lecture du fichier CSV
df_blackout_ml = pd.read_csv('df_blackout_ML_pop.csv', sep=';',low_memory=False)
# Lecture du fichier CSV
df_blackout_ml.head()

,region,date_heure,annee,jour_numero,heure,jour_fractionnel,thermique,nucleaire,eolien,solaire,...,nucleaire_p_disp,eolien_p_disp,solaire_p_disp,cap_prod_max_hydraulique,ind_prod_hydraulique,hydraulique_p_disp,cap_prod_max_bioenergies,bioenergies_p_disp,region_p_max,population
0,CENTRE VAL DE LOIRE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,90.0,9085.0,508.0,0.0,...,11630.0,662.0,207.0,91.0,0.0,91.0,41.0,41.0,12788.0,2570548
1,PAYS DE LA LOIRE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,127.0,0.0,182.0,0.0,...,0.0,454.0,385.0,5.0,0.0,5.0,57.0,57.0,2738.0,3660852
2,GRAND EST,2013-01-01 00:30:00,2013,1,00:30:00,1.02,319.0,9137.0,1109.0,0.0,...,10820.0,1803.0,498.0,2280.0,0.0,2280.0,79.0,79.0,18878.0,5552388
3,ILE DE FRANCE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,685.0,0.0,16.0,0.0,...,0.0,37.0,108.0,19.0,0.0,19.0,262.0,262.0,2547.0,11959807
4,OCCITANIE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,78.0,2497.0,367.0,0.0,...,2620.0,861.0,1040.0,5368.0,0.0,5368.0,126.0,126.0,10260.0,5683878


In [4]:
df_blackout_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121396 entries, 0 to 2121395
Data columns (total 29 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   region                    object 
 1   date_heure                object 
 2   annee                     int64  
 3   jour_numero               int64  
 4   heure                     object 
 5   jour_fractionnel          float64
 6   thermique                 float64
 7   nucleaire                 float64
 8   eolien                    float64
 9   solaire                   float64
 10  hydraulique               float64
 11  bioenergies               float64
 12  pompage                   float64
 13  ech_physiques             float64
 14  stockage_batterie         float64
 15  destockage_batterie       float64
 16  cap_prod_max_thermique    float64
 17  thermique_p_disp          float64
 18  cap_prod_max_nucleaire    float64
 19  nucleaire_p_disp          float64
 20  eolien_p_disp           

In [5]:
# Suppression des colonnes non nécessaires
col_to_drop = ['ind_prod_hydraulique','region_p_max']
df_blackout_ml = df_blackout_ml.drop(col_to_drop, axis=1)
df_blackout_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121396 entries, 0 to 2121395
Data columns (total 27 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   region                    object 
 1   date_heure                object 
 2   annee                     int64  
 3   jour_numero               int64  
 4   heure                     object 
 5   jour_fractionnel          float64
 6   thermique                 float64
 7   nucleaire                 float64
 8   eolien                    float64
 9   solaire                   float64
 10  hydraulique               float64
 11  bioenergies               float64
 12  pompage                   float64
 13  ech_physiques             float64
 14  stockage_batterie         float64
 15  destockage_batterie       float64
 16  cap_prod_max_thermique    float64
 17  thermique_p_disp          float64
 18  cap_prod_max_nucleaire    float64
 19  nucleaire_p_disp          float64
 20  eolien_p_disp           

In [6]:
# Sélection de 10 000 lignes aléatoires
df_random_sample = df_blackout_ml.sample(n=10000, random_state=1)

# Exportation des données échantillonnées
df_random_sample.to_csv('sample_10000.csv', index=False)

In [7]:
# Filtrage des données pour les années 2013 à 2019 (les années 2020 à 2023 seront utilisées pour vérifier les prédictions)
df_blackout_ml_filtered = df_blackout_ml[(df_blackout_ml['annee'] >= 2013) & (df_blackout_ml['annee'] <= 2019)].copy()

In [8]:
# Conversion de la colonne 'date_heure' en datetime
df_blackout_ml_filtered['date_heure'] = pd.to_datetime(df_blackout_ml_filtered['date_heure'])

# Conversion de l'heure et des minutes en heure décimale
df_blackout_ml_filtered['heure_decimale'] = df_blackout_ml_filtered['date_heure'].dt.hour + df_blackout_ml_filtered['date_heure'].dt.minute / 60

In [9]:
# Encodages cycliques des données temporelles
df_blackout_ml_filtered['heure_decimale_sin'] = np.sin(2 * np.pi * df_blackout_ml_filtered['heure_decimale'] / 24)
df_blackout_ml_filtered['heure_decimale_cos'] = np.cos(2 * np.pi * df_blackout_ml_filtered['heure_decimale'] / 24)

In [10]:
# Fonction pour vérifier si une année est bissextile
def bissextile(annee):
    return annee % 4 == 0 and (annee % 100 != 0 or annee % 400 == 0)

In [22]:
# Ajout d'une colonne pour le nombre total de jours dans l'année en fonction de l'année
df_blackout_ml_filtered['total_jours_annee'] = df_blackout_ml_filtered['annee'].apply(
    lambda annee: 366 if bissextile(annee) else 365)

In [23]:
# Calcul des encodages cycliques en fonction du nombre de jours dans l'année
df_blackout_ml_filtered['jour_fractionnel_sin'] = np.sin(2 * np.pi * df_blackout_ml_filtered['jour_fractionnel'] / df_blackout_ml_filtered['total_jours_annee'])
df_blackout_ml_filtered['jour_fractionnel_cos'] = np.cos(2 * np.pi * df_blackout_ml_filtered['jour_fractionnel'] / df_blackout_ml_filtered['total_jours_annee'])

In [24]:
# Suppression de la colonne 'total_jours_annee'
df_blackout_ml_filtered = df_blackout_ml_filtered.drop('total_jours_annee', axis=1)

In [25]:
# Division des données en ensembles d'entraînement et de test
features = df_blackout_ml_filtered[['region', 'annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                                    'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                                    'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                                    'cap_prod_max_nucleaire', 'nucleaire_p_disp','cap_prod_max_hydraulique', 'hydraulique_p_disp',
                                    'cap_prod_max_bioenergies','bioenergies_p_disp',
                                    'solaire_p_disp', 'eolien_p_disp', 'population']]

target = df_blackout_ml_filtered['ech_physiques']

In [26]:
# Encodage des variables catégorielles (comme 'region')
features = pd.get_dummies(features, columns=['region'])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [28]:
# Standardisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

features.head()

,annee,jour_fractionnel_sin,jour_fractionnel_cos,heure_decimale_sin,heure_decimale_cos,thermique,nucleaire,eolien,solaire,hydraulique,...,region_BRETAGNE,region_CENTRE VAL DE LOIRE,region_GRAND EST,region_HAUTS DE FRANCE,region_ILE DE FRANCE,region_NORMANDIE,region_NOUVELLE AQUITAINE,region_OCCITANIE,region_PAYS DE LA LOIRE,region_PROVENCE ALPES COTE D AZUR
0,2013,0.017558,0.999846,0.130526,0.991445,90.0,9085.0,508.0,0.0,34.0,...,False,True,False,False,False,False,False,False,False,False
1,2013,0.017558,0.999846,0.130526,0.991445,127.0,0.0,182.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,2013,0.017558,0.999846,0.130526,0.991445,319.0,9137.0,1109.0,0.0,1418.0,...,False,False,True,False,False,False,False,False,False,False
3,2013,0.017558,0.999846,0.130526,0.991445,685.0,0.0,16.0,0.0,0.0,...,False,False,False,False,True,False,False,False,False,False
4,2013,0.017558,0.999846,0.130526,0.991445,78.0,2497.0,367.0,0.0,943.0,...,False,False,False,False,False,False,False,True,False,False


In [29]:
# MACHINE LEARNING GLOBAL 

# Initialisation des modèles
models = {
    "Random Forest": RandomForestRegressor(n_estimators=30, random_state=42,n_jobs=None),
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet Regression": ElasticNet(),
    "Decision Tree": DecisionTreeRegressor(),    
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=50, random_state=42), 
    "AdaBoost": AdaBoostRegressor(n_estimators=60, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=60, random_state=42)
}

from tqdm import tqdm

# Création d'un DataFrame pour stocker les résultats
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'MAE', 'R2', 'Train Score', 'Test Score'])

# Entraînement et évaluation des modèles

for name, model in models.items() :
    print (f"Entraînement du modèle {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    
# Création d'un DataFrame temporaire pour la nouvelle ligne de résultats
    new_result = pd.DataFrame({
        'Model': [name], 
        'MSE': [mse], 
        'RMSE': [rmse], 
        'MAE': [mae], 
        'R2': [r2],
        'Train Score': [train_score],
        'Test Score': [test_score]
    })
    
    # Ajout des résultats
    results = pd.concat([results, new_result], ignore_index=True)
    # Affichage des résultats pour chaque modèle
    print(f"Modèle {name} terminé. MSE : {mse:.4f}, RMSE: {rmse:.4f},  MAE : {mae : .4f}, R2: {r2:.4f}, Train Score: {train_score:.4f},Test Score: {test_score : .4f}")
    
print("Tous les modèles ont été évalués.")

Entraînement du modèle Random Forest...
Modèle Random Forest terminé. MSE : 32755.7956, RMSE: 180.9856,  MAE :  123.7252, R2: 0.9983, Train Score: 0.9997,Test Score:  0.9983
Entraînement du modèle Linear Regression...
Modèle Linear Regression terminé. MSE : 471451.9821, RMSE: 686.6236,  MAE :  496.1775, R2: 0.9760, Train Score: 0.9763,Test Score:  0.9760
Entraînement du modèle Ridge Regression...
Modèle Ridge Regression terminé. MSE : 471452.8942, RMSE: 686.6243,  MAE :  496.1770, R2: 0.9760, Train Score: 0.9763,Test Score:  0.9760
Entraînement du modèle Lasso Regression...
Modèle Lasso Regression terminé. MSE : 473994.8921, RMSE: 688.4729,  MAE :  497.1765, R2: 0.9759, Train Score: 0.9762,Test Score:  0.9759
Entraînement du modèle ElasticNet Regression...
Modèle ElasticNet Regression terminé. MSE : 1247036.4773, RMSE: 1116.7079,  MAE :  818.3569, R2: 0.9366, Train Score: 0.9368,Test Score:  0.9366
Entraînement du modèle Decision Tree...
Modèle Decision Tree terminé. MSE : 73564.7694, 

In [30]:
# Exportation des résultats dans un fichier Excel
results.to_excel("models_ech_phys_results.xlsx", index=False, engine='openpyxl')

In [31]:
# MODELISATION PAR SEGMENTATION (Par région)

# Initialisation des modèles
models = {
    "Random Forest": RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=1),
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet Regression": ElasticNet(),
    "Decision Tree": DecisionTreeRegressor(),    
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=50, random_state=42), 
    "AdaBoost": AdaBoostRegressor(n_estimators=60, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=60, random_state=42)
}

# Liste des régions uniques
regions = df_blackout_ml_filtered['region'].unique()

# Dictionnaire pour stocker les résultats par région
results_by_region = {}

# Entraînement et évaluation des modèles par région
for region in regions:
    # Filtrage des données pour la région actuelle
    df_region = df_blackout_ml_filtered[df_blackout_ml_filtered['region'] == region].copy()
    
    # Séparation des variables explicatives de la cible
    features = df_region[['annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                          'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                          'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                          'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                          'cap_prod_max_bioenergies','bioenergies_p_disp',
                          'solaire_p_disp', 'eolien_p_disp', 'population']]
    
    target = df_region['ech_physiques']
    
    # Encodage des variables catégorielles (comme 'region') - ici, 'region' est déjà filtré
    features = pd.get_dummies(features)
    
    # Division des données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    # Standardisation des données
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Création d'un DataFrame pour stocker les résultats pour cette région
    results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'MAE', 'R2', 'Train Score', 'Test Score'])
    
    # Entraînement et évaluation des modèles
    for name, model in models.items():
        print(f"Entraînement du modèle {name} pour la région {region}...")  # Affiche le nom du modèle en cours
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calcul des métriques
        mse = mean_squared_error(y_test, y_pred)
        rmse = sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)
        
        # Création d'un DataFrame temporaire pour la nouvelle ligne de résultats
        new_result = pd.DataFrame({
            'Model': [name], 
            'MSE': [mse], 
            'RMSE': [rmse], 
            'MAE': [mae], 
            'R2': [r2],
            'Train Score': [train_score],
            'Test Score': [test_score]
        })
        
        # Ajout des résultats
        results = pd.concat([results, new_result], ignore_index=True)
        # Affichage des résultats pour chaque modèle
        print(f"Modèle {name} terminé pour la région {region}. MSE : {mse:.4f}, RMSE: {rmse:.4f},  MAE : {mae:.4f}, R2: {r2:.4f}, Train Score: {train_score:.4f}, Test Score: {test_score:.4f}")
    
    # Stockage des résultats dans le dictionnaire
    results_by_region[region] = results
    
    # Exportation des résultats pour chaque région dans un fichier Excel
    results.to_excel(f"models_ech_phys_results_{region}.xlsx", index=False, engine='openpyxl')

print("Tous les modèles ont été évalués pour chaque région.")

Entraînement du modèle Random Forest pour la région CENTRE VAL DE LOIRE...
Modèle Random Forest terminé pour la région CENTRE VAL DE LOIRE. MSE : 10363.2149, RMSE: 101.7999,  MAE : 74.5194, R2: 0.9957, Train Score: 0.9993, Test Score: 0.9957
Entraînement du modèle Linear Regression pour la région CENTRE VAL DE LOIRE...
Modèle Linear Regression terminé pour la région CENTRE VAL DE LOIRE. MSE : 80058.2231, RMSE: 282.9456,  MAE : 222.3356, R2: 0.9670, Train Score: 0.9669, Test Score: 0.9670
Entraînement du modèle Ridge Regression pour la région CENTRE VAL DE LOIRE...
Modèle Ridge Regression terminé pour la région CENTRE VAL DE LOIRE. MSE : 80058.0667, RMSE: 282.9453,  MAE : 222.3356, R2: 0.9670, Train Score: 0.9669, Test Score: 0.9670
Entraînement du modèle Lasso Regression pour la région CENTRE VAL DE LOIRE...
Modèle Lasso Regression terminé pour la région CENTRE VAL DE LOIRE. MSE : 80181.5080, RMSE: 283.1634,  MAE : 222.5635, R2: 0.9670, Train Score: 0.9669, Test Score: 0.9670
Entraînem